Lab to demonstrate the difference between a trusted and enabled FK, and the eventual performance impact.

Kernel: SQL, for any version of SQL Server since 2008

In [53]:
drop table table2
drop table table1 
GO
create table table1
(
	id int not null IDENTITY(1,1) CONSTRAINT PK_table1 PRIMARY KEY
,
	whatever varchar(100)
)
create table table2
(
	id int not null IDENTITY(1,1) CONSTRAINT PK_table2 PRIMARY KEY
,
	table1id int not null CONSTRAINT FK_table2_table1 FOREIGN KEY REFERENCES dbo.table1 (id) --No WITH CHECK needed here. Creates the FK trusted.
,
	whatever varchar(100)
)
GO
INSERT INTO table1
	(whatever)
values
	('abc')
GO
INSERT INTO table1
	(whatever)
select left(whatever + str(id),100)
from table1 
GO 15 --2^15 or 32,768 rows
GO
INSERT INTO [dbo].[table2]
	(table1id, whatever)
select id, whatever
from [dbo].[table1]
GO
SELECT
	Table_Name	= s.name + '.' +o.name 
, FK_Name		= fk.name 
, fk.is_not_trusted
, fk.is_disabled
FROM sys.foreign_keys as FK
	INNER JOIN sys.objects as o ON fk.parent_object_id = o.object_id
	INNER JOIN sys.schemas as s ON o.schema_id = s.schema_id
where o.name in ('table1','table2')


Commands completed successfully.

Total execution time: 00:00:00.0668498

Commands completed successfully.

Total execution time: 00:00:00.0931035

(1 row affected)

...

(32768 rows affected)

Total execution time: 00:00:13.4114274

(1 row affected)

Total execution time: 00:00:00.0530024

Table_Name,FK_Name,is_not_trusted,is_disabled
dbo.table2,FK_table2_table1,0,0


The origin of the problem is when you script out the FK with SSMS.

Let's reproduce by using the same code SSMS produces when you script out drop/creating a FK.

In [49]:
ALTER TABLE [dbo].[table2] DROP CONSTRAINT [FK_table2_table1]
GO
ALTER TABLE [dbo].[table2]  WITH NOCHECK ADD  CONSTRAINT [FK_table2_table1] FOREIGN KEY([table1id]) --Note the NOCHECK here. The FK is enabled but not Trusted.
REFERENCES [dbo].[table1] ([id])
GO
ALTER TABLE [dbo].[table2] CHECK CONSTRAINT [FK_table2_table1] --This doesn't re-trust the FK!
GO

SELECT  
	Table_Name	= s.name + '.' +o.name 
,	FK_Name		= fk.name 
,	fk.is_not_trusted
,	fk.is_disabled
FROM    sys.foreign_keys as FK
        INNER JOIN sys.objects as o ON fk.parent_object_id = o.object_id
        INNER JOIN sys.schemas as s ON o.schema_id = s.schema_id
where o.name in ('table1','table2')


Commands completed successfully.

Total execution time: 00:00:00.0517249

Commands completed successfully.

Total execution time: 00:00:00.0522722

Commands completed successfully.

Total execution time: 00:00:00.0503553

(1 row affected)

Total execution time: 00:00:00.1069140

Table_Name,FK_Name,is_not_trusted,is_disabled
dbo.table2,FK_table2_table1,1,0


Let's try to insert a value that shouldn't be allowed. 

The Insert is blocked even though the FK isn't trusted, because it is still enabled. We do get an error.


In [50]:
INSERT INTO [dbo].[table2] (table1id, whatever) 
OUTPUT inserted.table1id, inserted.whatever
VALUES (-1, 'whatever')
GO
SELECT  
	Table_Name	= s.name + '.' +o.name 
,	FK_Name		= fk.name 
,	fk.is_not_trusted
,	fk.is_disabled
FROM    sys.foreign_keys as FK
        INNER JOIN sys.objects as o ON fk.parent_object_id = o.object_id
        INNER JOIN sys.schemas as s ON o.schema_id = s.schema_id
where o.name in ('table1','table2')
GO


: Msg 547, Level 16, State 0, Line 1
The INSERT statement conflicted with the FOREIGN KEY constraint "FK_table2_table1". The conflict occurred in database "btrdynastyffl", table "dbo.table1", column 'id'.

The statement has been terminated.

Total execution time: 00:00:00.0645840

table1id,whatever


(1 row affected)

Total execution time: 00:00:00.0473966

Table_Name,FK_Name,is_not_trusted,is_disabled
dbo.table2,FK_table2_table1,1,0


The concept of disabling a FK vs not trusting a FK is important.

A disabled FK will allow invalid data into the child table at any time.

A FK can be created without trust so that it will allow EXISTING invalid data in the child table, but still block NEW invalid data in the child table.


In [51]:
ALTER TABLE [dbo].[table2] 
NOCHECK CONSTRAINT [FK_table2_table1] --Note: no WITH, just NOCHECK, this DISABLES the FK. 
GO
SELECT  
	Table_Name	= s.name + '.' +o.name 
,	FK_Name		= fk.name 
,	fk.is_not_trusted
,	fk.is_disabled
FROM    sys.foreign_keys as FK
        INNER JOIN sys.objects as o ON fk.parent_object_id = o.object_id
        INNER JOIN sys.schemas as s ON o.schema_id = s.schema_id
where o.name in ('table1','table2')
GO
--Try to insert a value that shouldn't be allowed
--The Insert is NOT blocked because the FK is disabled.
INSERT INTO [dbo].[table2] (table1id, whatever) 
OUTPUT inserted.table1id, inserted.whatever
VALUES (-1, 'whatever')
GO
ALTER TABLE [dbo].[table2] 
CHECK CONSTRAINT [FK_table2_table1] --Note: Enables the foreign key but does not mark it as trusted. Re-enabling the FK with invalid data in the child table still works!
GO
SELECT  
	Table_Name	= s.name + '.' +o.name 
,	FK_Name		= fk.name 
,	fk.is_not_trusted
,	fk.is_disabled
FROM    sys.foreign_keys as FK
        INNER JOIN sys.objects as o ON fk.parent_object_id = o.object_id
        INNER JOIN sys.schemas as s ON o.schema_id = s.schema_id
where o.name in ('table1','table2')
GO


Commands completed successfully.

Total execution time: 00:00:00.0514888

(1 row affected)

Total execution time: 00:00:00.0528005

(1 row affected)

...

(1 row affected)

Total execution time: 00:00:00.0495700

Table_Name,FK_Name,is_not_trusted,is_disabled
dbo.table2,FK_table2_table1,1,1


table1id,whatever
-1,whatever


Table_Name,FK_Name,is_not_trusted,is_disabled
dbo.table2,FK_table2_table1,1,0


Let's try to re-trust a FK while invalid records exist in the child table.

In [52]:
SELECT  
	Table_Name	= s.name + '.' +o.name 
,	FK_Name		= fk.name 
,	fk.is_not_trusted
,	fk.is_disabled
FROM    sys.foreign_keys as FK
        INNER JOIN sys.objects as o ON fk.parent_object_id = o.object_id
        INNER JOIN sys.schemas as s ON o.schema_id = s.schema_id
where o.name in ('table1','table2')
GO
ALTER TABLE [dbo].[table2] 
WITH CHECK  --Note: ATTEMPT to re-trust the foreign key. This command reverses the previous WITH NOCHECK. Re-trusting the FK with invalid data in the child table does NOT work! Will have to clean up data.
CHECK CONSTRAINT [FK_table2_table1] 
GO
DELETE FROM [dbo].[table2]  -- Clean up invalid data in the child table. In reality, you'll want to update invalid relations in the child table, or add new records in the parent table to make them valid. 
OUTPUT deleted.table1id, deleted.whatever
where table1id not in (select id from [dbo].[table1])
GO
ALTER TABLE [dbo].[table2] 
WITH CHECK  --Note: Re-trusts the foreign key. This is successful, and now the FK is trusted and enabled and can be used by SQL Server.
CHECK CONSTRAINT [FK_table2_table1] 
GO
SELECT  
	Table_Name	= s.name + '.' +o.name 
,	FK_Name		= fk.name 
,	fk.is_not_trusted
,	fk.is_disabled
FROM    sys.foreign_keys as FK
        INNER JOIN sys.objects as o ON fk.parent_object_id = o.object_id
        INNER JOIN sys.schemas as s ON o.schema_id = s.schema_id
where o.name in ('table1','table2')
GO

(1 row affected)

Total execution time: 00:00:00.0484057

: Msg 547, Level 16, State 0, Line 10
The ALTER TABLE statement conflicted with the FOREIGN KEY constraint "FK_table2_table1". The conflict occurred in database "btrdynastyffl", table "dbo.table1", column 'id'.

Total execution time: 00:00:00.0552781

(1 row affected)

...

(1 row affected)

Total execution time: 00:00:00.0482569

Table_Name,FK_Name,is_not_trusted,is_disabled
dbo.table2,FK_table2_table1,1,0


table1id,whatever
-1,whatever


Table_Name,FK_Name,is_not_trusted,is_disabled
dbo.table2,FK_table2_table1,0,0


What's the real impact of an untrusted FK? Let's use a simple query.

In [56]:
ALTER TABLE [dbo].[table2] DROP CONSTRAINT [FK_table2_table1]
GO
ALTER TABLE [dbo].[table2]  
WITH NOCHECK --Creates the Foreign Key but doesn't trust it! It's not enforced.
ADD  CONSTRAINT [FK_table2_table1] FOREIGN KEY([table1id])
REFERENCES [dbo].[table1] ([id])
GO
SELECT  
	Table_Name	= s.name + '.' +o.name 
,	FK_Name		= fk.name 
,	fk.is_not_trusted
,	fk.is_disabled
FROM    sys.foreign_keys as FK
        INNER JOIN sys.objects as o ON fk.parent_object_id = o.object_id
        INNER JOIN sys.schemas as s ON o.schema_id = s.schema_id
where o.name in ('table1','table2')
GO
SET SHOWPLAN_TEXT ON
GO
select table2.id  from table1 inner join table2 on table1.id = table2.table1id
GO
SET SHOWPLAN_TEXT OFF
GO


Commands completed successfully.

Total execution time: 00:00:00.0499100

Commands completed successfully.

Total execution time: 00:00:00.0499808

(1 row affected)

...

Commands completed successfully.

Total execution time: 00:00:00.0408895

Table_Name,FK_Name,is_not_trusted,is_disabled
dbo.table2,FK_table2_table1,1,0


StmtText
select table2.id from table1 inner join table2 on table1.id = table2.table1id


StmtText
"|--Hash Match(Inner Join, HASH:([btrdynastyffl].[dbo].[table1].[id])=([btrdynastyffl].[dbo].[table2].[table1id]))"
|--Clustered Index Scan(OBJECT:([btrdynastyffl].[dbo].[table1].[PK_table1]))
|--Clustered Index Scan(OBJECT:([btrdynastyffl].[dbo].[table2].[PK_table2]))


 Without a trusted FK (above), SQL can't assume the data is valid in both side of the inner join. Two clustered index scans.

 With a trusted FK (below), SQL can assume the data is valid in both side of the inner join. Skips the scan on table 1.

In [57]:
ALTER TABLE [dbo].[table2] 
WITH CHECK  --Note: Re-trusts the foreign key. This command reverses the previous WITH NOCHECK. Re-trusting the FK with invalid data in the child table does NOT work! Will have to clean up data.
CHECK CONSTRAINT [FK_table2_table1] 
GO

SELECT  
	Table_Name	= s.name + '.' +o.name 
,	FK_Name		= fk.name 
,	fk.is_not_trusted
,	fk.is_disabled
FROM    sys.foreign_keys as FK
        INNER JOIN sys.objects as o ON fk.parent_object_id = o.object_id
        INNER JOIN sys.schemas as s ON o.schema_id = s.schema_id
where o.name in ('table1','table2')
GO
SET SHOWPLAN_TEXT ON
GO
select table2.id  from table1 inner join table2 on table1.id = table2.table1id
GO
SET SHOWPLAN_TEXT OFF
GO


Commands completed successfully.

Total execution time: 00:00:00.6154876

(1 row affected)

Total execution time: 00:00:00.1066356

Commands completed successfully.

...

Commands completed successfully.

Total execution time: 00:00:00.0413188

Table_Name,FK_Name,is_not_trusted,is_disabled
dbo.table2,FK_table2_table1,0,0


StmtText
select table2.id from table1 inner join table2 on table1.id = table2.table1id


StmtText
|--Clustered Index Scan(OBJECT:([btrdynastyffl].[dbo].[table2].[PK_table2]))


Cleanup 

In [0]:
drop table dbo.table2 
drop table dbo.table1 
